### Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import requests 
import xlsxwriter 
import math 
import yfinance as yf

### Importing Data (list of static S&P500 stocks)

In [ ]:
stocks = pd.read_csv('sp_500_stocks.csv')

In [ ]:
print(stocks)

### Obtaining API

In [ ]:
# from secrets1 import AV_PRICE_API_TOKEN
# from secrets2 import AV_MKTCAP_API_TOKEN

### Calling one API

In [ ]:
# symbol='AAPL'
# api_price_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={AV_PRICE_API_TOKEN}'
# data_price = requests.get(api_price_url).json()
# data_price 

In [ ]:
# api_mktcap_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={AV_MKTCAP_API_TOKEN}'
# data_mktcap = requests.get(api_mktcap_url).json()
# data_mktcap

In [ ]:
# data_price['Global Quote']['05. price']
# data_mktcap['MarketCapitalization']

In [ ]:
# print(data_price['Global Quote']['05. price'])
# print(data_mktcap['MarketCapitalization'])

### Looping through Tickers using API

In [ ]:
# for symbol in stocks['Ticker'][:5]:
#     api_price_url = f'https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={symbol}&apikey={AV_PRICE_API_TOKEN}'
#     api_mktcap_url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol={symbol}&apikey={AV_MKTCAP_API_TOKEN}'
#     data_price = requests.get(api_price_url).json()
#     data_mktcap = requests.get(api_mktcap_url).json()
#     new_row = pd.DataFrame([[symbol, data_price['Global Quote']['05. price'], data_mktcap['MarketCapitalization'], 'N/A']],
#                        columns=my_columns)
#     df = pd.concat([df, new_row], ignore_index=True)

# df

### Looping through Tickers in S&P500 Stocks

In [ ]:
my_columns = ['Ticker', 'Price', 'Number Of Shares to Buy']
df = pd.DataFrame(columns = my_columns)
df

In [ ]:
for symbol in stocks['Ticker']:
    try:
        ticker = yf.Ticker(symbol)
        data = ticker.history()
        last_quote = data['Close'].iloc[-1]
        new_row = pd.DataFrame([[symbol, last_quote, 'N/A']], columns=my_columns)
        df = pd.concat([df, new_row], ignore_index=True)
    except Exception as e:
        continue
df

### Calculating number of shares to buy

In [ ]:
while True:
    portfolio_size = input("Please enter the value of your portfolio: ")
    
    try:
        val = float(portfolio_size)
        break  # Exit the loop if the input is a valid float
    except ValueError:
        print("That's not a number! Please try again.")

In [ ]:
position_size = float(portfolio_size) / len(df.index)
print("Position Size for each company is: $" f'{position_size}')

df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
for i in range(0, len(df.index)):
    df.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / df['Price'][i])
df

### Formatting Excel Output

In [ ]:
writer = pd.ExcelWriter('Equal_Weights_S&P500.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Recommended Trades', index = False)

In [ ]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [ ]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [ ]:
writer.close()

### Backtesting the Strategy

In [ ]:
import backtrader as bt

In [ ]:
class EqualWeightSP500(bt.Strategy):
    params = (
        ('rebalance_days', 60),  # Rebalance every 60 days
    )

    def __init__(self):
        self.counter = 0  # Count days to trigger rebalancing
        self.stocks = self.datas  # List of stock data feeds

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} {txt}')

    def next(self):
        # Increment counter each day
        self.counter += 1
        print(f"Day {self.counter}: Checking for rebalancing...")

        # Rebalance the portfolio every 30 days
        if self.counter % self.params.rebalance_days == 0:
            print("Rebalancing portfolio...")
            self.rebalance_portfolio()

    def rebalance_portfolio(self):
         # Get the value of the portfolio
        portfolio_value = self.broker.getvalue()

        # Equal weight: Calculate how much money to allocate per stock
        num_stocks = len(self.stocks)
        if num_stocks == 0:
            print("No stocks found for rebalancing.")
            return

        # Amount of money to allocate to each stock 
        position_size = portfolio_value / num_stocks
        
        # Sell all positions
        for data in self.stocks:
            position = self.getposition(data)
            if position.size > 0:
                print(f"Selling all shares of {data._name}")
                self.sell(data=data, size=position.size)
    
        # Buy equal shares for each stock
        for data in self.stocks:
            price = data.close[0] # Get the current closing price
            if price > 0: # Ensure valid price
                num_shares = math.floor(position_size / price)
                print(f"Buying {num_shares} shares of {data._name} at {price}")
                self.buy(data=data, size=num_shares)


In [ ]:
def load_stock_data(symbol, start_date, end_date):
    df = yf.download(f'{symbol}', start=start_date, end=end_date, interval='1d')
    df = df[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']]
    
    # Create a complete date range and reindex
    date_range = pd.date_range(start=start_date, end=end_date, freq='B')  # 'B' for business days
    df = df.reindex(date_range)
    
    # fill missing values after reindexing
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)

    # Ensure the shape after reindexing
    print(f"Data for {symbol}: {df.shape}")
    
    data = bt.feeds.PandasData(
        dataname=df, 
        fromdate=pd.Timestamp(start_date), 
        todate=pd.Timestamp(end_date)
    )
    data._name = symbol
    
    return data, df  # Return both Backtrader data feed and original DataFrame

In [ ]:
# Initialize Backtrader engine (Cerebro)
cerebro = bt.Cerebro()

# Initial capital for backtest
cerebro.broker.setcash(1000000)

In [ ]:
for stock in df['Ticker']:
    data, original_df = load_stock_data(stock, '2014-10-01', '2024-10-01')  # Load both data feed and DataFrame
    print(f"Data for {stock}: {original_df.shape}")  # Print the shape of the DataFrame
    cerebro.adddata(data)  # Add the stock data to Backtrader

In [ ]:
cerebro.addstrategy(EqualWeightSP500)

In [ ]:
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.Returns, _name='returns')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trades')

In [ ]:
print(f"Initial Portfolio Value: {cerebro.broker.getvalue()}")

results = cerebro.run()

print(f"Final Portfolio Value: {cerebro.broker.getvalue()}")

In [ ]:
# Accessing the results of analyzers
strat = results[0]  # Get the strategy instance

# Extract statistics from analyzers
sharpe = strat.analyzers.sharpe.get_analysis()
drawdown = strat.analyzers.drawdown.get_analysis()
returns = strat.analyzers.returns.get_analysis()
trades = strat.analyzers.trades.get_analysis()

# Print out or save the results
print(f'Sharpe Ratio: {sharpe["sharperatio"]}')
print(f'Max Drawdown: {drawdown["max"]["drawdown"]}%')
print(f'Annual Return: {returns["rnorm100"]}%')

# Accessing trade statistics
print(f'Total Trades: {trades.total.total}')
print(f'Winning Trades: {trades.won.total}')
print(f'Losing Trades: {trades.lost.total}')

### Visualling Backtesting Results

In [ ]:
import matplotlib.pyplot as plt
from IPython.display import display, Javascript, Image
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = [15, 12]
plt.rcParams.update({'font.size': 12})

In [ ]:
fig = cerebro.plot(iplot=False)[0][0]
fig.savefig('backtest_results.png')  # Save to file
Image('backtest_results.png')